### Libraries

In [1]:
import geopandas as gpd
import pandas as pd

### Loading Data

In [2]:
route = '/home/nacho/Desktop/data/'

seccions = [
    4815075, 4815078, 4815079, 4815081, 4815083, 4815084, 4815085, 4815086, 4815087,
    4815090, 4815091, 4815096, 4815097, 4815098, 4815099, 4815425, 4815428, 4815429,
    4815440, 4815441
]

# Dictionary to store the GeoDataFrames
street_info_dict = {}

for sec in seccions:
    geojson_file_path_street = rf'{route}STREETS_NAV/SREETS_NAV_{sec}.geojson'
    try:
        street_info_dict[sec] = gpd.read_file(geojson_file_path_street)
    except Exception as e:
        print(f"Error reading file for section {sec}: {e}")

In [3]:
street_name_dict = {}

for sec in seccions:
    geojson_file_path_street_name = rf'{route}STREETS_NAMING_ADDRESSING/SREETS_NAMING_ADDRESSING_{sec}.geojson'
    try:
        street_name_dict[sec] = gpd.read_file(geojson_file_path_street_name)
    except Exception as e:
        print(f"Error reading file for section {sec}: {e}")

In [4]:
merged_dict = {}

for sec in seccions:
    try:
        df_info = street_info_dict[sec]
        df_name = street_name_dict[sec]
        
        merged = df_info.merge(df_name, on='geometry', suffixes=('_info', '_name'))
        merged_dict[sec] = merged
        
    except Exception as e:
        print(f"Error merging section {sec}: {e}")

In [5]:
df_4815075 = merged_dict[4815075]
df_4815078 = merged_dict[4815078]
df_4815079 = merged_dict[4815079]
df_4815081 = merged_dict[4815081]
df_4815083 = merged_dict[4815083]
df_4815084 = merged_dict[4815084]
df_4815085 = merged_dict[4815085]
df_4815086 = merged_dict[4815086]
df_4815087 = merged_dict[4815087]
df_4815090 = merged_dict[4815090]
df_4815091 = merged_dict[4815091]
df_4815096 = merged_dict[4815096]
df_4815097 = merged_dict[4815097]
df_4815098 = merged_dict[4815098]
df_4815099 = merged_dict[4815099]
df_4815425 = merged_dict[4815425]
df_4815428 = merged_dict[4815428]
df_4815429 = merged_dict[4815429]
df_4815440 = merged_dict[4815440]
df_4815441 = merged_dict[4815441]


In [6]:
pois_dic = {}

for sec in seccions:
    try:
        pois_dic[sec] = pd.read_csv(rf'{route}POIs/POI_{sec}.csv')
    except Exception as e:
        print(f"Error reading POI file for section {sec}: {e}")

/tmp/ipykernel_41308/2173714624.py:5: DtypeWarning: Columns (9,10,27) have mixed types. Specify dtype option on import or set low_memory=False.
  pois_dic[sec] = pd.read_csv(rf'{route}POIs/POI_{sec}.csv')
/tmp/ipykernel_41308/2173714624.py:5: DtypeWarning: Columns (27,35) have mixed types. Specify dtype option on import or set low_memory=False.
  pois_dic[sec] = pd.read_csv(rf'{route}POIs/POI_{sec}.csv')


## Data Cleaning process

lets analyze the first df just to confirm 

In [7]:
df_4815075

,AR_AUTO,AR_BUS,AR_CARPOOL,AR_DELIV,AR_EMERVEH,AR_MOTOR,AR_PEDEST,AR_TAXIS,AR_TRAFF,AR_TRUCKS,...,R_ADDRSCH,R_NREFADDR,R_REFADDR,ST_LANGCD,ST_NAME,ST_NM_BASE,ST_NM_SUFF,ST_TYP_AFT,ST_TYP_ATT,ST_TYP_BEF
0,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,E,672,30,SPA,CALLE MARIANO ZÚÑIGA,MARIANO ZÚÑIGA,None,None,N,CALLE
1,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,O,101,125,SPA,CALLE CARLOTA,CARLOTA,None,None,N,CALLE
2,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,None,None,None,None,None,None,None,None,N,None
3,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,None,None,None,None,None,None,None,None,N,None
4,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,O,121,139,SPA,CALLE ANA MARÍA,ANA MARÍA,None,None,N,CALLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31201,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,None,None,None,None,None,None,None,None,N,None
31202,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,None,None,None,SPA,AVENIDA SAN ISIDRO,SAN ISIDRO,None,None,N,AVENIDA
31203,Y,Y,Y,Y,Y,Y,N,Y,Y,Y,...,None,None,None,SPA,AUTOPISTA TOLUCA-NAUCALPAN,TOLUCA-NAUCALPAN,None,None,N,AUTOPISTA
31204,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,...,None,None,None,None,None,None,None,None,N,None


In [8]:
df_4815075.columns

Index(['AR_AUTO', 'AR_BUS', 'AR_CARPOOL', 'AR_DELIV', 'AR_EMERVEH', 'AR_MOTOR',
       'AR_PEDEST', 'AR_TAXIS', 'AR_TRAFF', 'AR_TRUCKS', 'BRIDGE', 'CONTRACC',
       'COVERIND', 'DIR_TRAVEL', 'DIVIDER', 'FERRY_TYPE', 'FROM_LANES',
       'FRONTAGE', 'FR_SPD_LIM', 'FUNC_CLASS', 'INDESCRIB', 'INTERINTER',
       'LANE_CAT', 'link_id_info', 'LOW_MBLTY', 'MANOEUVRE', 'MULTIDIGIT',
       'PAVED', 'POIACCESS', 'PRIORITYRD', 'PRIVATE', 'PUB_ACCESS', 'RAMP',
       'ROUNDABOUT', 'SPEED_CAT', 'TOLLWAY', 'TO_LANES', 'TO_SPD_LIM',
       'TUNNEL', 'UNDEFTRAFF', 'URBAN', 'geometry', 'ADDR_TYPE',
       'link_id_name', 'L_ADDRFORM', 'L_ADDRSCH', 'L_NREFADDR', 'L_REFADDR',
       'R_ADDRFORM', 'R_ADDRSCH', 'R_NREFADDR', 'R_REFADDR', 'ST_LANGCD',
       'ST_NAME', 'ST_NM_BASE', 'ST_NM_SUFF', 'ST_TYP_AFT', 'ST_TYP_ATT',
       'ST_TYP_BEF'],
      dtype='object')

* para poder hacer el merge, cambiemos el nombre de link_id_info a LINK_ID (el mismo que el de pois)

In [9]:
df_4815075.rename(columns = {"link_id_info": "LINK_ID"}, inplace= True)

In [10]:
df_4815075["MULTIDIGIT"].value_counts()

MULTIDIGIT
N    29082
Y     2124
Name: count, dtype: int64

In [11]:
# lets just keep with the links which are multidigit == Y for each data
df_4815075 = df_4815075[df_4815075["MULTIDIGIT"] == "Y"]

In [12]:
df_4815075[df_4815075["LINK_ID"] == 1296526969]

,AR_AUTO,AR_BUS,AR_CARPOOL,AR_DELIV,AR_EMERVEH,AR_MOTOR,AR_PEDEST,AR_TAXIS,AR_TRAFF,AR_TRUCKS,...,R_ADDRSCH,R_NREFADDR,R_REFADDR,ST_LANGCD,ST_NAME,ST_NM_BASE,ST_NM_SUFF,ST_TYP_AFT,ST_TYP_ATT,ST_TYP_BEF
306,Y,Y,Y,N,Y,Y,Y,Y,Y,N,...,None,None,None,SPA,AVENIDA SOLIDARIDAD LAS TORRES,SOLIDARIDAD LAS TORRES,None,None,N,AVENIDA


* ahora llamemos el df de los POIs

In [13]:
df_pois_4815075 = pois_dic[4815075]

In [14]:
columns_pois = ["LINK_ID", "POI_ID", "FAC_TYPE", "POI_NAME", "POI_ST_NUM"]

In [15]:
df_pois_4815075 = df_pois_4815075[columns_pois]
df_pois_4815075.head(4)

,LINK_ID,POI_ID,FAC_TYPE,POI_NAME,POI_ST_NUM
0,1296526969,1244439551,4013,TOLUCA CENTRO,NaN
1,702722866,1244248545,9535,MISCELÁNEA,144.0
2,702722866,1178939983,9535,TIENDA DE ABARROTES,NaN
3,1296526966,1244944824,4013,TOLUCA CENTRO,NaN


In [16]:
df_pois_4815075.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7232 entries, 0 to 7231
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   LINK_ID     7232 non-null   int64  
 1   POI_ID      7232 non-null   int64  
 2   FAC_TYPE    7232 non-null   int64  
 3   POI_NAME    7232 non-null   object 
 4   POI_ST_NUM  676 non-null    float64
dtypes: float64(1), int64(3), object(1)
memory usage: 282.6+ KB


In [17]:
df_pois_4815075[df_pois_4815075["LINK_ID"] == 1296526969]

,LINK_ID,POI_ID,FAC_TYPE,POI_NAME,POI_ST_NUM
0,1296526969,1244439551,4013,TOLUCA CENTRO,NaN


In [18]:
df_4815075[df_4815075["LINK_ID"] == 1296526969]

,AR_AUTO,AR_BUS,AR_CARPOOL,AR_DELIV,AR_EMERVEH,AR_MOTOR,AR_PEDEST,AR_TAXIS,AR_TRAFF,AR_TRUCKS,...,R_ADDRSCH,R_NREFADDR,R_REFADDR,ST_LANGCD,ST_NAME,ST_NM_BASE,ST_NM_SUFF,ST_TYP_AFT,ST_TYP_ATT,ST_TYP_BEF
306,Y,Y,Y,N,Y,Y,Y,Y,Y,N,...,None,None,None,SPA,AVENIDA SOLIDARIDAD LAS TORRES,SOLIDARIDAD LAS TORRES,None,None,N,AVENIDA


In [19]:
# hacemos el merge
prueba = df_4815075.merge(df_pois_4815075, on="LINK_ID", how="left")


In [20]:
prueba[prueba["LINK_ID"] == 702722866]

,AR_AUTO,AR_BUS,AR_CARPOOL,AR_DELIV,AR_EMERVEH,AR_MOTOR,AR_PEDEST,AR_TAXIS,AR_TRAFF,AR_TRUCKS,...,ST_NAME,ST_NM_BASE,ST_NM_SUFF,ST_TYP_AFT,ST_TYP_ATT,ST_TYP_BEF,POI_ID,FAC_TYPE,POI_NAME,POI_ST_NUM
49,Y,Y,Y,Y,Y,Y,Y,Y,Y,N,...,CALLE GLORIA,GLORIA,None,None,N,CALLE,1.244249e+09,9535.0,MISCELÁNEA,144.0
50,Y,Y,Y,Y,Y,Y,Y,Y,Y,N,...,CALLE GLORIA,GLORIA,None,None,N,CALLE,1.178940e+09,9535.0,TIENDA DE ABARROTES,NaN


In [21]:
prueba.columns

# Carga el archivo con los POIs
poi_link_data = pd.read_csv(rf'{route}POIs/POI_4815075.csv')

# Verifica las columnas de ambos DataFrames
print("Columnas de prueba antes del merge:", prueba.columns)
print("Columnas de poi_link_data:", poi_link_data.columns)

# Renombra o elimina la columna conflictiva en 'prueba' si existe
if "POI_ID" in prueba.columns:
    prueba = prueba.drop(columns=["POI_ID"])  # Elimina la columna conflictiva

# Realiza el merge con un subconjunto de columnas
prueba = prueba.merge(poi_link_data[["LINK_ID", "POI_ID"]], on="LINK_ID", how="left")

# Verifica las columnas después del merge
print("Columnas después del merge:", prueba.columns)

# Filtra para dejar solo los LINK_IDs que tienen al menos un POI_ID
prueba = prueba[prueba["POI_ID"].notnull()]
prueba.columns

Columnas de prueba antes del merge: Index(['AR_AUTO', 'AR_BUS', 'AR_CARPOOL', 'AR_DELIV', 'AR_EMERVEH', 'AR_MOTOR',
       'AR_PEDEST', 'AR_TAXIS', 'AR_TRAFF', 'AR_TRUCKS', 'BRIDGE', 'CONTRACC',
       'COVERIND', 'DIR_TRAVEL', 'DIVIDER', 'FERRY_TYPE', 'FROM_LANES',
       'FRONTAGE', 'FR_SPD_LIM', 'FUNC_CLASS', 'INDESCRIB', 'INTERINTER',
       'LANE_CAT', 'LINK_ID', 'LOW_MBLTY', 'MANOEUVRE', 'MULTIDIGIT', 'PAVED',
       'POIACCESS', 'PRIORITYRD', 'PRIVATE', 'PUB_ACCESS', 'RAMP',
       'ROUNDABOUT', 'SPEED_CAT', 'TOLLWAY', 'TO_LANES', 'TO_SPD_LIM',
       'TUNNEL', 'UNDEFTRAFF', 'URBAN', 'geometry', 'ADDR_TYPE',
       'link_id_name', 'L_ADDRFORM', 'L_ADDRSCH', 'L_NREFADDR', 'L_REFADDR',
       'R_ADDRFORM', 'R_ADDRSCH', 'R_NREFADDR', 'R_REFADDR', 'ST_LANGCD',
       'ST_NAME', 'ST_NM_BASE', 'ST_NM_SUFF', 'ST_TYP_AFT', 'ST_TYP_ATT',
       'ST_TYP_BEF', 'POI_ID', 'FAC_TYPE', 'POI_NAME', 'POI_ST_NUM'],
      dtype='object')
Columnas de poi_link_data: Index(['Unnamed: 0', 'LINK_ID', '

Index(['AR_AUTO', 'AR_BUS', 'AR_CARPOOL', 'AR_DELIV', 'AR_EMERVEH', 'AR_MOTOR',
       'AR_PEDEST', 'AR_TAXIS', 'AR_TRAFF', 'AR_TRUCKS', 'BRIDGE', 'CONTRACC',
       'COVERIND', 'DIR_TRAVEL', 'DIVIDER', 'FERRY_TYPE', 'FROM_LANES',
       'FRONTAGE', 'FR_SPD_LIM', 'FUNC_CLASS', 'INDESCRIB', 'INTERINTER',
       'LANE_CAT', 'LINK_ID', 'LOW_MBLTY', 'MANOEUVRE', 'MULTIDIGIT', 'PAVED',
       'POIACCESS', 'PRIORITYRD', 'PRIVATE', 'PUB_ACCESS', 'RAMP',
       'ROUNDABOUT', 'SPEED_CAT', 'TOLLWAY', 'TO_LANES', 'TO_SPD_LIM',
       'TUNNEL', 'UNDEFTRAFF', 'URBAN', 'geometry', 'ADDR_TYPE',
       'link_id_name', 'L_ADDRFORM', 'L_ADDRSCH', 'L_NREFADDR', 'L_REFADDR',
       'R_ADDRFORM', 'R_ADDRSCH', 'R_NREFADDR', 'R_REFADDR', 'ST_LANGCD',
       'ST_NAME', 'ST_NM_BASE', 'ST_NM_SUFF', 'ST_TYP_AFT', 'ST_TYP_ATT',
       'ST_TYP_BEF', 'FAC_TYPE', 'POI_NAME', 'POI_ST_NUM', 'POI_ID'],
      dtype='object')

In [24]:
from shapely.geometry import LineString
from shapely.ops import nearest_points
from rtree import index
import numpy as np

# Función para calcular la dirección de un segmento de línea
def calculate_direction(coord1, coord2):
    dx = coord2[0] - coord1[0]
    dy = coord2[1] - coord1[1]
    angle = np.arctan2(dy, dx)  # Ángulo en radianes
    return angle

# Función para verificar si dos líneas son paralelas
def are_lines_parallel(line1, line2, tolerance=0.1):
    if isinstance(line1, LineString) and isinstance(line2, LineString):
        coords1 = list(line1.coords)
        coords2 = list(line2.coords)
        
        # Calcular direcciones de segmentos en ambas líneas
        directions1 = [calculate_direction(coords1[i], coords1[i + 1]) for i in range(len(coords1) - 1)]
        directions2 = [calculate_direction(coords2[i], coords2[i + 1]) for i in range(len(coords2) - 1)]
        
        # Comparar las direcciones promedio de ambas líneas
        avg_direction1 = np.mean(directions1)
        avg_direction2 = np.mean(directions2)
        
        # Verificar si las direcciones están dentro del rango de tolerancia
        return abs(avg_direction1 - avg_direction2) < tolerance
    return False

# Función para calcular la distancia mínima entre dos líneas
def calculate_distance(line1, line2):
    if isinstance(line1, LineString) and isinstance(line2, LineString):
        p1, p2 = nearest_points(line1, line2)
        return p1.distance(p2)
    return None

# Función optimizada para encontrar líneas paralelas entre diferentes LINK_IDs
def find_parallel_link_ids_with_distances_optimized(df, tolerance=0.1):
    parallel_links = []
    spatial_index = index.Index()

    # Crear un índice espacial para las geometrías
    for idx, row in df.iterrows():
        spatial_index.insert(idx, row["geometry"].bounds)

    # Comparar solo geometrías cercanas
    for idx1, row1 in df.iterrows():
        possible_matches_index = list(spatial_index.intersection(row1["geometry"].bounds))
        for idx2 in possible_matches_index:
            # Verificar que idx2 sea válido
            if idx2 >= len(df) or idx2 < 0:
                continue
            if idx1 >= idx2:  # Evitar comparaciones duplicadas
                continue
            row2 = df.iloc[idx2]
            if row1["LINK_ID"] != row2["LINK_ID"]:  # Comparar diferentes LINK_IDs
                if are_lines_parallel(row1["geometry"], row2["geometry"], tolerance):
                    distance = calculate_distance(row1["geometry"], row2["geometry"])
                    parallel_links.append({
                        "LINK_ID_1": row1["LINK_ID"],
                        "LINK_ID_2": row2["LINK_ID"],
                        "distance": distance
                    })
    return parallel_links

# Llamar a la función corregida
parallel_links_with_distances = find_parallel_link_ids_with_distances_optimized(prueba, tolerance=0.1)

# Crear un conjunto de todos los LINK_IDs que tienen líneas paralelas
parallel_link_ids = set()
for item in parallel_links_with_distances:
    parallel_link_ids.add(item["LINK_ID_1"])
    parallel_link_ids.add(item["LINK_ID_2"])

# Identificar los LINK_IDs que no son paralelos con ningún otro
all_link_ids = set(prueba["LINK_ID"].unique())
non_parallel_link_ids = all_link_ids - parallel_link_ids

# Imprimir los resultados
print("LINK_IDs que no son paralelos con ningún otro:")
for link_id in non_parallel_link_ids:
    print(f"LINK_ID {link_id}")

print('LINK_IDs que no son paralelos con ningún otro:', len(non_parallel_link_ids))

LINK_IDs que no son paralelos con ningún otro:
LINK_ID 702656000
LINK_ID 1191296002
LINK_ID 1041600516
LINK_ID 847943186
LINK_ID 702665243
LINK_ID 937985053
LINK_ID 938938398
LINK_ID 920545310
LINK_ID 798007839
LINK_ID 1159100449
LINK_ID 720474146
LINK_ID 1179630115
LINK_ID 1159100450
LINK_ID 1336588325
LINK_ID 938938401
LINK_ID 1244987943
LINK_ID 1336719400
LINK_ID 721416233
LINK_ID 1336719402
LINK_ID 702964777
LINK_ID 1328569387
LINK_ID 1328569388
LINK_ID 1347012654
LINK_ID 721416239
LINK_ID 1328569389
LINK_ID 1336588328
LINK_ID 1159378482
LINK_ID 1159378483
LINK_ID 1337541173
LINK_ID 1343805494
LINK_ID 1159378485
LINK_ID 792517688
LINK_ID 1347051065
LINK_ID 847922234
LINK_ID 1296154683
LINK_ID 1347915836
LINK_ID 847922235
LINK_ID 847922237
LINK_ID 1347915839
LINK_ID 1348537914
LINK_ID 1347051068
LINK_ID 1242500671
LINK_ID 1242500674
LINK_ID 702729795
LINK_ID 1196922855
LINK_ID 1179766854
LINK_ID 847922246
LINK_ID 721672778
LINK_ID 1346381899
LINK_ID 1346381900
LINK_ID 1346234443
LIN

In [26]:
# Filtrar los LINK_IDs paralelos que cumplen con las condiciones de FRONTAGE y DIR_TRAVEL
filtered_parallel_links = []

for item in parallel_links_with_distances:
    link_id_1 = item["LINK_ID_1"]
    link_id_2 = item["LINK_ID_2"]
    
    # Obtener las filas correspondientes a los LINK_IDs
    row1 = prueba[prueba["LINK_ID"] == link_id_1]
    row2 = prueba[prueba["LINK_ID"] == link_id_2]
    
    # Verificar las condiciones de FRONTAGE y DIR_TRAVEL
    if not row1.empty and not row2.empty:
        if row1.iloc[0]["FRONTAGE"] == row2.iloc[0]["FRONTAGE"] and row1.iloc[0]["DIR_TRAVEL"] == row2.iloc[0]["DIR_TRAVEL"]:
            filtered_parallel_links.append(item)

# Crear un DataFrame con los resultados filtrados
filtered_parallel_links_df = pd.DataFrame(filtered_parallel_links)

# Crear un conjunto de LINK_IDs únicos a partir de los resultados filtrados
unique_filtered_link_ids = set(filtered_parallel_links_df["LINK_ID_1"]).union(
    set(filtered_parallel_links_df["LINK_ID_2"])
)

# Mostrar los resultados corregidos
print("Número de LINK_IDs únicos en los resultados filtrados:")
print(unique_filtered_link_ids)
print(len(unique_filtered_link_ids))

Número de LINK_IDs únicos en los resultados filtrados:
{1073188352, 1336243968, 1115310850, 702992387, 1358112261, 1358112262, 1358112263, 1073188359, 1346455560, 1358112266, 702702090, 1346455563, 702657550, 1346455566, 1346455567, 1208943380, 1208943381, 1346733852, 1346733853, 702702109, 1295413532, 702661408, 1347012645, 1347012646, 702678312, 1347012652, 1295327533, 1199169583, 1347012657, 1347012659, 1347051062, 938804024, 847905081, 938804025, 1346736699, 1347915840, 1208943429, 711775046, 1201356871, 1201356872, 1175857993, 1201356874, 1201356875, 711775054, 1045028949, 848189271, 1045028952, 1201356889, 1376771417, 1376771418, 1346378845, 1295327583, 1296128099, 1296128100, 920544873, 1296128107, 1346876782, 1175918196, 1037681015, 708766584, 1041601407, 1041601410, 1159135875, 1159135876, 1372810370, 1201428104, 702693514, 1245007755, 798662551, 1044375197, 1296128165, 702711207, 1296128170, 1296128176, 1347045812, 1346480309, 1346480308, 1296128189, 1179841472, 702677954, 11

### LINK_ID que son Incorrect Multiply Digitised attribution

In [ ]:
# Combinar los LINK_IDs únicos filtrados con los non_parallel_link_ids
all_unique_link_ids = unique_filtered_link_ids.union(non_parallel_link_ids)

# Mostrar los resultados finales
print("Número total de LINK_IDs únicos (incluyendo paralelos y no paralelos):")
print(all_unique_link_ids)
print(len(all_unique_link_ids))



Número total de LINK_IDs únicos (incluyendo paralelos y no paralelos):
{np.int32(1191296002), 702992387, np.int32(1041600516), 1346455560, 1346455563, 702657550, 1346455566, 1346455567, np.int32(937985053), np.int32(938938398), np.int32(920545310), np.int32(1159100449), np.int32(720474146), np.int32(1159100450), np.int32(938938401), 1347012645, 1347012646, np.int32(1336588325), np.int32(1336719400), np.int32(721416233), np.int32(1336719402), np.int32(702964777), 1347012652, np.int32(1328569387), np.int32(1328569388), 1199169583, np.int32(1347012654), 1347012657, np.int32(721416239), 1347012659, np.int32(1328569389), np.int32(1343805494), np.int32(792517688), np.int32(847922234), np.int32(1296154683), np.int32(1347915836), np.int32(847922235), np.int32(847922237), np.int32(1347915839), 1347915840, np.int32(1179766854), 1201356871, 1201356872, np.int32(847922246), 1201356874, 1201356875, np.int32(1346381899), np.int32(1346381900), np.int32(1346234443), np.int32(1346381904), np.int32(1346

### Para LINK_IDs que son No POI in reality

In [ ]:
def classify_non_existent_pois(df_mm_sections):
    non_existent_results = {}

    for sec, df in df_mm_sections.items():
        # Filtramos solo las filas con al menos un POI válido
        df_filtered = df[df["POI_ID"].notna()].copy()

        # Creamos columna vacía para la clasificación
        df_filtered["POI_STATUS"] = "Valid"
        df_filtered["MARKED_FOR_DELETION"] = False


        # Caso 1: ST_TYP_BEF ≠ ST_TYP_AFT
        cond_type_changed = df_filtered["ST_TYP_BEF"] != df_filtered["ST_TYP_AFT"]

        # Caso 2: MULTIDIGIT + (BRIDGE o TUNNEL o RAMP)
        cond_infra_issue = (
            (df_filtered["MULTIDIGIT"] == "Y") &
            (
                (df_filtered["BRIDGE"] == "Y") |
                (df_filtered["TUNNEL"] == "Y") |
                (df_filtered["RAMP"] == "Y")
            )
        )

        # Aplicamos ambas condiciones
        df_filtered.loc[cond_type_changed | cond_infra_issue, "POI_STATUS"] = "Non Existent POI"
        df_filtered.loc[cond_non_existent, "MARKED_FOR_DELETION"] = True

        # Guardamos resultados por sección
        non_existent_results[sec] = df_filtered

    return non_existent_results

classified_poi_dict = classify_non_existent_pois(prueba)